In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,352 kB]
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,057 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,253 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main a

In [2]:
# Import packages
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, max, min, col, isnan, when, count, round, year
import time

In [3]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

In [5]:
# Create a temporary view of the DataFrame.
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True, ignoreLeadingWhiteSpace=True)

# Display DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# Check Null values 
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
| id|date|date_built|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
|  0|   0|         0|    0|       0|        0|          0|       0|     0|         0|   0|
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+



In [7]:
# Print out datatypes
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [8]:
# Convert datatypes
df = df\
    .withColumn("date", df["date"].cast('date'))\
    .withColumn("date_built", df["date_built"].cast('integer'))\
    .withColumn("price", df["price"].cast("integer"))\
    .withColumn("bedrooms", df["bedrooms"].cast("integer"))\
    .withColumn("bathrooms", df["bathrooms"].cast("integer"))\
    .withColumn("sqft_living", df["sqft_living"].cast("integer"))\
    .withColumn("sqft_lot", df["sqft_lot"].cast("integer"))\
    .withColumn("floors", df["floors"].cast("integer"))\
    .withColumn("waterfront", df["waterfront"].cast("integer"))\
    .withColumn("view", df["view"].cast("integer"))

In [9]:
# Print updated datatypes
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [10]:
# Add column 'Year' to original DataFrame
df = df.withColumn("year",year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [11]:
# Create a temporary view. The temp view is a pre-requisite for Spark to allow you to read with sql.  
df.createOrReplaceTempView('home_sales')

In [12]:
# What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
    (SELECT year, round(avg(price),2) AS average_price
    FROM home_sales
    WHERE bedrooms == 4
    GROUP BY year
    ORDER BY year)
"""

spark.sql(query).show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [13]:
# What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms 
# rounded to two decimal places?
query = """
    (SELECT date_built, round(avg(price),2) AS average_price 
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3
    GROUP BY date_built
    ORDER BY date_built)
"""

spark.sql(query).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
    (SELECT date_built, round(avg(price),2) AS average_price 
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built)
"""

spark.sql(query).show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, 
# where the homes are greater than or equal to $350,000? Although this is a small dataset, 
#determine the run time for this query.
 
start_time = time.time()

query = """
    (SELECT round(avg(view),2) as average_view
    FROM home_sales
    WHERE price >= 350000)
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.484266996383667 seconds ---


In [16]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime. See Summary below.  

start_time = time.time()

query = """
    (SELECT round(avg(view),2) as average_view
    FROM home_sales
    WHERE price >= 350000)
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.4020729064941406 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [20]:
# 11. Read the formatted parquet data.
p_df_p = spark.read.parquet('home_sales_partitioned')

In [21]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('parq_home_sales')

In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. See Summary below.  

start_time = time.time()

query = """
    (SELECT round(avg(view),2) as average_view
    FROM parq_home_sales
    WHERE price >= 350000)
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.49326205253601074 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [24]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False